In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import mouse_utils as mu
import torch
from tqdm import tqdm

In [2]:
df = pd.read_csv("./data/K-Data.csv")

v1 = df.query("region == 'V1'")
m = v1.m.unique()[2]
v1 = v1[v1.m == m] # take for all mice later
v1 = v1.copy()  # to prevent warning
v1["mouse_unit"] = v1["m"] + "_" + v1["u"].astype(str)
v1 = v1.groupby(["mouse_unit", "grat_orientation", "grat_contrast", "grat_spat_freq", "grat_phase"]).mean().reset_index()
v1 = v1[["mouse_unit", "grat_orientation", "grat_contrast", "grat_spat_freq", "grat_phase", "response"]]

unique_units = v1['mouse_unit'].unique()
unique_orientation = v1['grat_orientation'].unique()
unique_contrast = v1['grat_contrast'].unique()
unique_spat_freq = v1['grat_spat_freq'].unique()
unique_phase = v1['grat_phase'].unique()

shape = (len(unique_units), len(unique_orientation), len(unique_contrast), len(unique_spat_freq), len(unique_phase))
result_array = np.full(shape, np.nan)

# Iterate through the DataFrame and fill the array
for index, row in tqdm(v1.iterrows()):
    u_index = np.where(unique_units == row['mouse_unit'])[0][0]
    orientation_index = np.where(unique_orientation == row['grat_orientation'])[0][0]
    contrast_index = np.where(unique_contrast == row['grat_contrast'])[0][0]
    spat_freq_index = np.where(unique_spat_freq == row['grat_spat_freq'])[0][0]
    phase_index = np.where(unique_phase == row['grat_phase'])[0][0]
    result_array[u_index, orientation_index, contrast_index, spat_freq_index, phase_index] = row['response']

result_array = np.mean(np.mean(result_array, axis=4), axis=3)
result_array = result_array.transpose((0, 2, 1))
result_array = result_array * 1000

result_array = torch.tensor(result_array)

/var/folders/78/9j91h4q516z7774553xzp19m0000gn/T/ipykernel_2779/3872892955.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/K-Data.csv")
/var/folders/78/9j91h4q516z7774553xzp19m0000gn/T/ipykernel_2779/3872892955.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  v1 = v1.groupby(["mouse_unit", "grat_orientation", "grat_contrast", "grat_spat_freq", "grat_phase"]).mean().reset_index()
186624it [00:04, 43238.65it/s]


In [3]:

# # J_array = [1.99, 1.9, 1.01, 0.79]  # Need to make parameters of an exponential
# # P_array = [0.11, 0.11, 0.45, 0.45]
# # w_array = [32., 32., 32., 32.]


# # J_array = [0.69, 0.64, 0., -0.29] # Max log values
# # P_array = [-2.21, -2.21, -0.8, -0.8]
# # w_array = [3.46, 3.46, 3.46, 3.46]

# J_array = [0, -0.29, 0.69, -0.64] # Keen log values
# P_array = [-0.8, -2.21, -2.21, -0.8]
# w_array = [3.46, 3.46, 3.46, 3.46]

# # J_array = [-5.1707, -0.0277,  0.9482,  0.1585]
# # P_array = [-6.1361, -1.9772,  0.8548,  1.2384]
# # w_array = [0.3990, 3.6245, 4.1893, 3.6955]

# # J_array = [0.69, 0.64, 0., -0.29]
# # P_array = [-1.21, -1.21, -0.8, -0.8]
# # w_array = [3.46, 3.46, 3.46, 3.46]

# # J_array = [9.04e-02, 3.82e-05, 7.62e-5, 2.52]
# # P_array = [1.93e-02,  8.78,  1.79e-04,  2.85]
# # w_array = [8.78, 166, 1.23e-2, 1.81e2]


# # J_array = [-1.5734, -0.20967, 0.472169, -0.6607]  # JAX minima 
# # P_array = [-2.2163,  -2.2553,  -2.4399,  -0.8935]
# # w_array = [2.054, 3.5095, 3.3344, 3.47027]




# model = mu.NeuroNN(J_array, P_array, w_array, 4000)
# if torch.cuda.is_available():
#     model = model.to('cuda')
#     print("Model moved to GPU.")
# else:
#     print("GPU not available. Keeping the model on CPU.")

# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# mu.training_loop(model, optimizer, result_array)

In [4]:
J_array = [0, -0.29, 0.69, -0.64] # Keen log values
P_array = [-0.8, -2.21, -2.21, -0.8]
w_array = [3.46, 3.46, 3.46, 3.46]


model = mu.NeuroNN(J_array, P_array, w_array, 10000)
if torch.cuda.is_available():
    model = model.to('cuda')
    print("Model moved to GPU.")
else:
    print("GPU not available. Keeping the model on CPU.")

mu.training_loop_no_backwards(model, result_array)

GPU not available. Keeping the model on CPU.
Generated a W matrix
